## Create TICKIT schema with some tables 
- it takes less than 1 minute to load this small dataset 
- [tickit sample data](https://docs.aws.amazon.com/redshift/latest/dg/c_sampledb.html)

In [0]:
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS venue;
DROP TABLE IF EXISTS category;
DROP TABLE IF EXISTS date;
DROP TABLE IF EXISTS event;
DROP TABLE IF EXISTS sales;

create table users(
	userid integer not null,
	username char(8),
	firstname varchar(30),
	lastname varchar(30),
	city varchar(30),
	state char(2),
	email varchar(100),
	phone char(14),
	likesports boolean,
	liketheatre boolean,
	likeconcerts boolean,
	likejazz boolean,
	likeclassical boolean,
	likeopera boolean,
	likerock boolean,
	likevegas boolean,
	likebroadway boolean,
	likemusicals boolean);

create table venue(
	venueid smallint not null,
	venuename varchar(100),
	venuecity varchar(30),
	venuestate char(2),
	venueseats integer);

create table category(
	catid smallint not null,
	catgroup varchar(10),
	catname varchar(10),
	catdesc varchar(50));

create table dwdate(
	dateid smallint not null,
	caldate date not null,
	day character(3) not null,
	week smallint not null,
	month character(5) not null,
	qtr character(5) not null,
	year smallint not null,
	holiday boolean default('N'));

create table event(
	eventid integer not null,
	venueid smallint not null,
	catid smallint not null,
	dateid smallint not null,
	eventname varchar(200),
	starttime timestamp);

create table listing(
	listid integer not null,
	sellerid integer not null,
	eventid integer not null,
	dateid smallint not null,
	numtickets smallint not null,
	priceperticket decimal(8,2),
	totalprice decimal(8,2),
	listtime timestamp);


create table sales(
	salesid integer not null,
	listid integer not null,
	sellerid integer not null,
	buyerid integer not null,
	eventid integer not null,
	dateid smallint not null,
	qtysold smallint not null,
	pricepaid decimal(8,2),
	commission decimal(8,2),
	saletime timestamp);

## COPY data from S3 
[COPY command](https://docs.aws.amazon.com/redshift/latest/dg/t_loading-tables-from-s3.html)

In [0]:
copy users from 's3://cdk-entest-videos/tickit/allusers_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy venue from 's3://cdk-entest-videos/tickit/venue_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy category from 's3://cdk-entest-videos/tickit/category_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy dwdate from 's3://cdk-entest-videos/tickit/date2008_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy event from 's3://cdk-entest-videos/tickit/allevents_pipe.txt'
iam_role default
delimiter '|' timeformat 'YYYY-MM-DD HH:MI:SS' region 'us-east-1';

copy listing from 's3://cdk-entest-videos/tickit/listings_pipe.txt'
iam_role default
delimiter '|' region 'us-east-1';

copy sales from 's3://cdk-entest-videos/tickit/sales_tab.txt'
iam_role default
delimiter '\t' timeformat 'MM/DD/YYYY HH:MI:SS' region 'us-east-1';

## System Table - Query History
[system tables](https://docs.aws.amazon.com/redshift/latest/dg/cm_chap_system-tables.html)

In [0]:
select * from SYS_QUERY_HISTORY;

## System Table - Load Data History

In [0]:
select * from SYS_LOAD_HISTORY;

## System Tables - Data Distribution

In [0]:
select * from svv_table_info;

## Simple Join 
- Find top 5 sellers in San Diego, based on the number of tickets sold in 2008

In [0]:
select sellerid, username, (firstname ||' '|| lastname) as name,
city, sum(qtysold)
from sales, dwdate, users
where sales.sellerid = users.userid
and sales.dateid = dwdate.dateid
and year = 2008
and city = 'San Diego'
group by sellerid, username, name, city
order by 5 desc
limit 5;